In [24]:
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [25]:
def load_data(file_name):
    file_path = Path(file_name).resolve()
    # Read the single CSV that has a 'step' column
    df = pd.read_csv(file_path)
    # Group rows by 'step' and turn each group into a list of records
    sheet_dicts = {
        f"Step {int(step)}": grp.to_dict(orient="records")
        for step, grp in df.groupby('step')
    }
    # Extract step-specific data (or empty list if missing)
    step1_data = sheet_dicts.get("Step 1", [])
    step2_data = sheet_dicts.get("Step 2", [])
    return sheet_dicts, step1_data, step2_data

def split_vendor_data(data):
    vendor_names    = [entry['vendor']     for entry in data]
    production_dict = {entry['vendor']: entry['production'] for entry in data}
    cost_dict       = {entry['vendor']: entry['cost']       for entry in data}
    return vendor_names, production_dict, cost_dict


In [26]:
#TODO: think about units
#TODO: build it as a class
#TODO: pyomo set
'''
def optimize(vendors, production, cost, demand):
    model = pyo.ConcreteModel(doc="flowsheet Optimization Model")
    steps = list(vendors.keys())
    index_set = [(g, o) for g in steps for o in vendors[g]]
    # Define binary decision variable
    model.x = pyo.Var(index_set, domain=pyo.Binary)

    # Constraint: select exactly one option per group
    def one_per_group_rule(model, g):
        return sum(model.x[g, o] for o in vendors[g]) == 1
    model.one_per_group = pyo.Constraint(steps, rule=one_per_group_rule)
    
    # Constraint: production level
    def prod_constraint(model):
        return sum(production[g][o] * model.x[g, o] for g in production for o in production[g]) >= demand
    model.prod_level = pyo.Constraint(rule=prod_constraint)

    # Objective: minimize cost
    def budget_constraint(model):
        return sum(production[g][o] * model.x[g, o] for g in production for o in production[g])
    model.obj = pyo.Objective(rule=budget_constraint, sense = pyo.minimize)

    
    opt = pyo.SolverFactory("gurobi")
    opt_success = opt.solve(model)

    model.pprint()'''

'\ndef optimize(vendors, production, cost, demand):\n    model = pyo.ConcreteModel(doc="flowsheet Optimization Model")\n    steps = list(vendors.keys())\n    index_set = [(g, o) for g in steps for o in vendors[g]]\n    # Define binary decision variable\n    model.x = pyo.Var(index_set, domain=pyo.Binary)\n\n    # Constraint: select exactly one option per group\n    def one_per_group_rule(model, g):\n        return sum(model.x[g, o] for o in vendors[g]) == 1\n    model.one_per_group = pyo.Constraint(steps, rule=one_per_group_rule)\n    \n    # Constraint: production level\n    def prod_constraint(model):\n        return sum(production[g][o] * model.x[g, o] for g in production for o in production[g]) >= demand\n    model.prod_level = pyo.Constraint(rule=prod_constraint)\n\n    # Objective: minimize cost\n    def budget_constraint(model):\n        return sum(production[g][o] * model.x[g, o] for g in production for o in production[g])\n    model.obj = pyo.Objective(rule=budget_constraint, 

In [27]:
def optimize(vendors, production, cost, demand):
    model = pyo.ConcreteModel()

    # 1) steps
    model.steps = pyo.Set(initialize=vendors.keys())

    # 2) a flat 2-D Set of (step, option) tuples
    pairs = [(g,o) for g in vendors for o in vendors[g]]
    model.step_option = pyo.Set(dimen=2, initialize=pairs)

    # 3) data as Params, indexed by that 2-D Set
    model.production = pyo.Param(
        model.step_option,
        initialize={(g,o): production[g][o] for g,o in pairs}
    )
    model.cost = pyo.Param(
        model.step_option,
        initialize={(g,o): cost[g][o]       for g,o in pairs}
    )

    # 4) binary decision vars on the same flat Set
    model.x = pyo.Var(model.step_option, domain=pyo.Binary)

    # 5) exactly one option per step
    def one_per_group(m, g):
        return sum(m.x[g,o] for o in vendors[g]) == 1
    model.one_per_group = pyo.Constraint(model.steps, rule=one_per_group)

    # 6) meet total demand
    model.demand = pyo.Param(initialize=demand)
    def prod_level(m):
        return sum(m.production[g,o]*m.x[g,o] for g,o in m.step_option) >= m.demand
    model.prod_level = pyo.Constraint(rule=prod_level)

    # 7) minimize total cost
    def obj_rule(m):
        return sum(m.cost[g,o]*m.x[g,o] for g,o in m.step_option)
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # 8) solve
    solver = pyo.SolverFactory("gurobi")
    solver.solve(model, tee=False)
    return model


In [28]:
sheet_dicts, step1_data, step2_data = load_data("test-data-combo.csv")
vendor_names1, production1, cost1 = split_vendor_data(step1_data)
vendor_names2, production2, cost2 = split_vendor_data(step2_data)

vendors = {"Step 1": vendor_names1, "Step 2": vendor_names2}
production = {"Step 1": production1,   "Step 2": production2}
cost       = {"Step 1": cost1,         "Step 2": cost2}

model = optimize(vendors, production, cost, demand=200)

model.pprint()

2 Set Declarations
    step_option : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :    6 : {('Step 1', 'A'), ('Step 1', 'B'), ('Step 2', 'A'), ('Step 2', 'B'), ('Step 2', 'C'), ('Step 2', 'D')}
    steps : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Step 1', 'Step 2'}

3 Param Declarations
    cost : Size=6, Index=step_option, Domain=Any, Default=None, Mutable=False
        Key             : Value
        ('Step 1', 'A') :  30.0
        ('Step 1', 'B') : 45.45
        ('Step 2', 'A') :  30.0
        ('Step 2', 'B') : 45.45
        ('Step 2', 'C') : 150.0
        ('Step 2', 'D') :  25.0
    demand : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   200
    production : Size=6, Index=step_option, Domain=Any, Default=None, Mutable=False
        Key             : Value
        ('Step 1', 'A') 